In [7]:
import sys, os, importlib, requests
import rasterio, geojson

import pandas as pd
import geopandas as gpd

from rasterstats import zonal_stats

import ee
ee.Initialize()

sys.path.append("/home/wb411133/Code/GEE_Zonal/src")
import gee_tools
import gee_helpers

sys.path.append("/home/wb411133/Code/gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

In [2]:
base_folder = "/home/wb411133/data/Country/AFG/"

In [3]:
admin_zones = '/home/wb411133/data/Country/AFG/ADMIN/GeoBoundaries/geoBoundaries-AFG-ADM1.shp'
military_bases = '/home/wb411133/data/Country/AFG/MilitaryBases/fake_bases.shp'
ntl_zonal_folder = '/home/wb411133/data/Country/AFG/NTL'
ntl_zonal = os.path.join(ntl_zonal_folder, f'{os.path.basename(admin_zones)[:-4]}_NTL.csv')

inD = gpd.read_file(admin_zones)
inB = gpd.read_file(military_bases)

ee_features = gee_helpers.gpd_to_gee(inD, 'PROV_34_NA')

In [4]:
s3_base = 's3://wbgdecinternal-ntl/'
ntl_file_list = "/home/wb411133/temp/YEM/AWS_NTL_S3.txt"
focal_tile = "TILE3"
temp_folder = "/home/wb411133/temp"

all_files = []
with open(ntl_file_list, 'r') as in_aws:
    for line in in_aws:
        if focal_tile in line and 'avg_rade9' in line:
            all_files.append(os.path.join(s3_base, line.split(" ")[-1][:-1]))
            
all_files[:5]

['s3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201204/TILE3/SVDNB_npp_20120401-20120430_75N060E_vcmcfg_v10_c201605121456.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201205/TILE3/SVDNB_npp_20120501-20120531_75N060E_vcmcfg_v10_c201605121458.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201206/TILE3/SVDNB_npp_20120601-20120630_75N060E_vcmcfg_v10_c201605121459.avg_rade9h.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201207/TILE3/SVDNB_npp_20120701-20120731_75N060E_vcmcfg_v10_c201605121509.avg_rade9.tif',
 's3://wbgdecinternal-ntl/NTL/VIIRS_UNZIP/viirs_201208/TILE3/SVDNB_npp_20120801-20120831_75N060E_vcmcfg_v10_c201602121348.avg_rade9.tif']

In [5]:
#if not os.path.exists(ntl_zonal):
for cur_tif in all_files[:5]:
    res = rMisc.zonalStats(inD, cur_tif, minVal=0.05)
    res = pd.DataFrame(res,columns=['SUM','MIN','MAX','MEAN'])
    inD[cur_tif.split("/")[5]] = res['SUM']
    tPrint(os.path.basename(cur_tif))

#    pd.DataFrame(inD.drop(['geometry'], axis=1)).to_csv(ntl_zonal)

11:01:40	SVDNB_npp_20120401-20120430_75N060E_vcmcfg_v10_c201605121456.avg_rade9h.tif
11:01:57	SVDNB_npp_20120501-20120531_75N060E_vcmcfg_v10_c201605121458.avg_rade9h.tif
11:02:13	SVDNB_npp_20120601-20120630_75N060E_vcmcfg_v10_c201605121459.avg_rade9h.tif
11:02:31	SVDNB_npp_20120701-20120731_75N060E_vcmcfg_v10_c201605121509.avg_rade9.tif
11:02:47	SVDNB_npp_20120801-20120831_75N060E_vcmcfg_v10_c201602121348.avg_rade9.tif


In [12]:
for cur_tif in all_files[:5]:
    zonal_stats(inD, cur_tif, geojson_out=True, all_touched=True)
    tPrint(os.path.basename(cur_tif))


/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  wa

11:06:59	SVDNB_npp_20120401-20120430_75N060E_vcmcfg_v10_c201605121456.avg_rade9h.tif


/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  wa

11:07:54	SVDNB_npp_20120501-20120531_75N060E_vcmcfg_v10_c201605121458.avg_rade9h.tif


/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  wa

11:09:22	SVDNB_npp_20120601-20120630_75N060E_vcmcfg_v10_c201605121459.avg_rade9h.tif


/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  wa

11:10:36	SVDNB_npp_20120701-20120731_75N060E_vcmcfg_v10_c201605121509.avg_rade9.tif


/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  wa

11:11:32	SVDNB_npp_20120801-20120831_75N060E_vcmcfg_v10_c201602121348.avg_rade9.tif


In [11]:
from rasterstats import gen_zonal_stats
gen_zonal_stats?